In [25]:
GLOBA_PATH = input()

https://physiology-consultation-belongs-administrator.trycloudflare.com


In [26]:
import time
import yaml

import pandas as pd
import requests


HEALTH_PATH = GLOBA_PATH + "/health"
GET_RECO_PATH = GLOBA_PATH +  "/reco/als/{user_id}"
GET_EXPLAIN_PATH = GLOBA_PATH + "/explain/als/{user_id}/{item_id}"

with open('../service/envs/authentication_env.yaml') as env_config:
    ENV_TOKEN = yaml.safe_load(env_config)
    
interaction = pd.read_csv("../service/data/kion_train/interactions.csv")
items = pd.read_csv("../service/data/kion_train/items.csv")

In [27]:
response = requests.get(HEALTH_PATH, headers={"Authorization": f"Bearer {ENV_TOKEN['token']}"}).json()
response

'I am alive'

1. Возьмем **пять юзеров**, которые **попали в онлайн кэш* и для них **объясним топ-10 рекомендации**
2. Возьмем **пять юзеров не из онлайн кэша** и для них **объясним топ-10 рекомендации** (Они будут одинаковыми, т.к. сервис будет выдавать популярное)

In [28]:
def get_reco(user_id):
    path_reco = GET_RECO_PATH.format(user_id=user_id)
    response = requests.get(path_reco, headers={"Authorization": f"Bearer {ENV_TOKEN['token']}"}).json()
    return response

def get_explanation(user_id, item_id):
    path_explain = GET_EXPLAIN_PATH.format(user_id=user_id, item_id=item_id)
    response = requests.get(path_explain, headers={"Authorization": f"Bearer {ENV_TOKEN['token']}"}).json()
    return response

# Пользователи, которые попали в кеш обучения

In [31]:
users_id = interaction["user_id"].sample(5).tolist()
users_id

[20500, 756638, 842575, 266055, 973305]

In [32]:
for user_id in users_id:
    reco = get_reco(user_id)
    print(f"=== Reco top-10: {reco['items']}")
    for item_id in reco["items"]:
        explanation = get_explanation(user_id, item_id)
        title = items[items["item_id"] == item_id]["title"].iloc[0]
        print(f"= Explanation for '{title}' item_id = {item_id}")
        print(f"= Процент релевантности: {explanation['score']}")
        print(f"= Объяснение: {explanation['explanation']}")
        print("--------------------------------------------------\n")
    print("********************************************************\n")

=== Reco top-10: [16166, 3182, 5411, 16270, 7582, 10761, 11749, 13018, 13243, 15266]
= Explanation for 'Зверополис' item_id = 16166
= Процент релевантности: 32
= Объяснение: Рекомендую, тем кому нравится 'Моана'
--------------------------------------------------

= Explanation for 'Ральф против Интернета' item_id = 3182
= Процент релевантности: 28
= Объяснение: Рекомендую, тем кому нравится 'Моана'
--------------------------------------------------

= Explanation for 'Монстры на каникулах 3: Море зовёт' item_id = 5411
= Процент релевантности: 19
= Объяснение: Рекомендую, тем кому нравится 'Моана'
--------------------------------------------------

= Explanation for 'Тайна Коко' item_id = 16270
= Процент релевантности: 18
= Объяснение: Рекомендую, тем кому нравится 'Моана'
--------------------------------------------------

= Explanation for 'Холодное сердце II' item_id = 7582
= Процент релевантности: 17
= Объяснение: Рекомендую, тем кому нравится 'Моана'
-------------------------------

=== Reco top-10: [4880, 4151, 6809, 10440, 9728, 7107, 13018, 849, 14741, 12995]
= Explanation for 'Афера' item_id = 4880
= Процент релевантности: 101
= Объяснение: Рекомендую, тем кому нравится 'Девятаев'
--------------------------------------------------

= Explanation for 'Секреты семейной жизни' item_id = 4151
= Процент релевантности: 101
= Объяснение: Рекомендую, тем кому нравится 'Девятаев'
--------------------------------------------------

= Explanation for 'Дуров' item_id = 6809
= Процент релевантности: 100
= Объяснение: Рекомендую, тем кому нравится 'Девятаев'
--------------------------------------------------

= Explanation for 'Хрустальный' item_id = 10440
= Процент релевантности: 100
= Объяснение: Рекомендую, тем кому нравится 'Девятаев'
--------------------------------------------------

= Explanation for 'Гнев человеческий' item_id = 9728
= Процент релевантности: 99
= Объяснение: Рекомендую, тем кому нравится 'Афера'
--------------------------------------------------

= 

# Пользователи, которых не было в обучении

In [33]:
users_id = [-1, -2, -3, -4, -5]

for user_id in users_id:
    print(user_id in interaction["user_id"])

False
False
False
False
False


In [34]:
for user_id in users_id:
    reco = get_reco(user_id)
    print(f"=== Reco top-10: {reco['items']}")
    for item_id in reco["items"]:
        explanation = get_explanation(user_id, item_id)
        title = items[items["item_id"] == item_id]["title"].iloc[0]
        print(f"= Explanation for '{title}' item_id = {item_id}")
        print(f"= Процент релевантности: {explanation['score']}")
        print(f"= Объяснение: {explanation['explanation']}")
        print("--------------------------------------------------\n")
    print("********************************************************\n")

=== Reco top-10: [2788, 3480, 3750, 11566, 1714, 16216, 13395, 856, 3642, 1914]
= Explanation for 'Ганга' item_id = 2788
= Процент релевантности: 93
= Объяснение: Рекомендую посмотреть контент жанра 'драмы, зарубежные'
--------------------------------------------------

= Explanation for 'Жизнь Мехэк' item_id = 3480
= Процент релевантности: 88
= Объяснение: Рекомендую посмотреть контент жанра 'зарубежные, мелодрамы'
--------------------------------------------------

= Explanation for 'Юлия Высоцкая' item_id = 3750
= Процент релевантности: 71
= Объяснение: Это нравится большому количеству пользователей
--------------------------------------------------

= Explanation for 'Татьянин день' item_id = 11566
= Процент релевантности: 95
= Объяснение: Рекомендую посмотреть контент жанра 'русские, мелодрамы'
--------------------------------------------------

= Explanation for 'Люби меня таким' item_id = 1714
= Процент релевантности: 94
= Объяснение: Рекомендую посмотреть контент жанра 'зарубеж

= Explanation for 'Неравный брак' item_id = 1914
= Процент релевантности: 92
= Объяснение: Рекомендую посмотреть контент жанра 'драмы, русские'
--------------------------------------------------

********************************************************

=== Reco top-10: [2788, 3480, 3750, 11566, 1714, 16216, 13395, 856, 3642, 1914]
= Explanation for 'Ганга' item_id = 2788
= Процент релевантности: 83
= Объяснение: Рекомендую посмотреть контент жанра 'драмы, зарубежные'
--------------------------------------------------

= Explanation for 'Жизнь Мехэк' item_id = 3480
= Процент релевантности: 91
= Объяснение: Рекомендую посмотреть контент жанра 'зарубежные, мелодрамы'
--------------------------------------------------

= Explanation for 'Юлия Высоцкая' item_id = 3750
= Процент релевантности: 73
= Объяснение: Это нравится большому количеству пользователей
--------------------------------------------------

= Explanation for 'Татьянин день' item_id = 11566
= Процент релевантности: 89
= Объя